In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/nlp-getting-started'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


I am going to use tensorflow library for Natural Language Processing

In [22]:
# Importing libraries

import tensorflow as tf

Importing the data using pandas

In [23]:
# Storing the data 

traindata = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
testdata = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [24]:
# Storing train data

train_data_list = traindata['text'].tolist()
train_labels = np.array(traindata['target'].tolist())

# Storing test data

evaluate_data_list = testdata['text'].tolist()

We need to convert english words into array. Tokenizer will help me, as it has function which will give me the vocab size.

In [25]:
# creating a function to store in suitable form

def nlp_store(data):

    token = tf.keras.preprocessing.text.Tokenizer()
    token.fit_on_texts(data)

    # storing the vocab size for padding
    vocab_size = len(token.word_index) + 1
    train_data = token.texts_to_sequences(data)
    train_data = np.array(train_data)
    
    return train_data, vocab_size

In [26]:
# Storing in suitable form

train_data, vocab_size = nlp_store(train_data_list)
evaluate_data, vocab_none = nlp_store(evaluate_data_list)

/tmp/ipykernel_2354/2644733524.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data = np.array(train_data)


In [27]:
evaluate_data

array([list([34, 567, 5, 985, 135, 100]),
       list([272, 53, 247, 11, 1907, 1103, 743, 890, 429]),
       list([75, 11, 5, 130, 42, 22, 891, 1908, 4022, 23, 4023, 986, 4, 568, 10, 1276, 481, 101, 43]),
       ..., list([714, 299, 732, 7, 914, 3, 1, 2, 12817]),
       list([3758, 697, 469, 379, 2469, 2470, 3, 1, 2, 12818]),
       list([3412, 58, 3419, 277, 2396, 71, 264, 2464])], dtype=object)

In [28]:
vocab_size

22701

Each tweet contains maximum 4000 characters. So, I need to create a padding sequence which will store only 100 words

In [29]:
# padding

train_data = tf.keras.utils.pad_sequences(train_data,
                                          100)
evaluate_data = tf.keras.utils.pad_sequences(evaluate_data,
                                        100)

To build my model, I will use Sequential, Long Short Term Memory(LSTM), Embeddings, and final dense layer for output.

In [30]:
# creating model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          726432    
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 734,785
Trainable params: 734,785
Non-trainable params: 0
_________________________________________________________________


Compiling the model and fitting the data

In [32]:
model.compile(loss = 'BinaryCrossentropy',
             optimizer = 'adam',
             metrics = ['acc'])

history = model.fit(train_data,
                   train_labels,
                   epochs = 20,
                   validation_split = 0.1)

Epoch 1/20
215/215 [==============================] - 19s 76ms/step - loss: 0.5578 - acc: 0.7095 - val_loss: 0.4523 - val_acc: 0.7913
Epoch 2/20
215/215 [==============================] - 4s 21ms/step - loss: 0.3149 - acc: 0.8739 - val_loss: 0.4642 - val_acc: 0.8005
Epoch 3/20
215/215 [==============================] - 3s 14ms/step - loss: 0.1790 - acc: 0.9368 - val_loss: 0.5697 - val_acc: 0.7638
Epoch 4/20
215/215 [==============================] - 3s 13ms/step - loss: 0.1018 - acc: 0.9656 - val_loss: 0.5496 - val_acc: 0.7717
Epoch 5/20
215/215 [==============================] - 2s 11ms/step - loss: 0.0621 - acc: 0.9806 - val_loss: 0.6802 - val_acc: 0.7730
Epoch 6/20
215/215 [==============================] - 2s 11ms/step - loss: 0.0406 - acc: 0.9886 - val_loss: 0.7345 - val_acc: 0.7638
Epoch 7/20
215/215 [==============================] - 2s 11ms/step - loss: 0.0310 - acc: 0.9915 - val_loss: 0.6894 - val_acc: 0.7795
Epoch 8/20
215/215 [==============================] - 2s 9ms/step - 

Predicting the value and storing the value in either 1 or 0

In [33]:
prediction = model.predict(evaluate_data)

pred = np.where(prediction > 0.5, 1, 0)
pred

102/102 [==============================] - 1s 3ms/step


array([[1],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]])

Exporting the testdata values

In [34]:
testdata["target"] = pred
submission = testdata.drop(['keyword', 'location', 'text'], axis = 1)
submission.to_csv("submission.csv", index=False)